# Accessing the Temperature Control Laboratory

## Importing `tclab`

Once installed the package can be imported into Python and an instance created with the Python statements

    import tclab
    lab = tclab.TCLab()

`tclab.setup()` creates an instance of a class the provides access to temperature measurements, heaters, and led on board the Temperature Control Laboratory. When called with no arguments,  attempts to find a device connected to a serial port and returns a connection. An error is generated if no device is found.  The connection should be closed with

    lab.close()
    
when no longer in use. The following cell demonstrates this process, and uses the tclab `LED()` function to flash the LED on the Temperature Control Lab for a period of 10 seconds at a 100% brightness level. 

In [ ]:
import tclab

lab = tclab.TCLab()
lab.LED(100)
lab.close()

## Using TCLab with Python's `with` statement

The Python `with` statement provides a simple means of setting up and closing a connection to the Temperature Control Laboratory. The with statement establishes a context where a tclab instance is created, assigned to a variable, and automatically closed upon completion.

In [ ]:
import tclab

with tclab.TCLab() as lab:
    lab.LED(100)

The `with` statement is likely to be the most common way to connect the Temperature Control Laboratory for most uses.

## Reading Temperatures

Once a tclab instance is created and connected to a device, the temperature sensors on the temperature control lab can be acccessed with the attributes `.T1` and `.T2`.  For example, given an instance `lab`, the temperatures are accessed as

    T1 = lab.T1
    T2 = a.T2

Note that `lab.T1` and `lab.T2` are read-only properties. Any attempt to set them to a value will return a Python error.

In [ ]:
import tclab

with tclab.TCLab() as lab:
    print("Temperature 1: {0:0.2f} °C".format(lab.T1))
    print("Temperature 2: {0:0.2f} °C".format(lab.T2))

## Setting Heaters

The heaters are controlled by functions `.Q1()` and `.Q2()` of a tclab instance. For example, both heaters can be set to 100% power with the functions

    lab.Q1(100)
    lab.Q2(100)

The device firmware limits the heaters to a range of 0 to 100%. The current value of attributes may be accessed via

    Q1 = lab.Q1()
    Q2 = lab.Q2()
    
Note that the retrieved values may be different due to the range-limiting enforced by the device firmware.

Alternatively, the heaters can also be specified with the properties `.U1` and `.U2`. Thus setting

    lab.U1 = 100
    lab.U2 = 100
    
would set both heaters to 100% power. The current value of the heaters can be accessed as

    print("Current setting of heater 1 is", lab.U1, "%")
    print("Current setting of heater 2 is", lab.U2, "%")
    
The choice to use a function (i.e, `.Q1()` and `.Q2()`) or a property (i.e, `.U1` or `.U2`) to set and access heater settings is a matter of user preference.

In [ ]:
import tclab
import time

with tclab.TCLab() as lab:
    print("\nStarting Temperature 1: {0:0.2f} °C".format(lab.T1),flush=True)
    print("Starting Temperature 2: {0:0.2f} °C".format(lab.T2),flush=True)

    lab.Q1(100)
    lab.Q2(100)
    print("\nSet Heater 1:", lab.Q1(), "%",flush=True)
    print("Set Heater 2:", lab.Q2(), "%",flush=True)
    
    t_heat = 30
    print("\nHeat for", t_heat, "seconds")
    time.sleep(t_heat)

    print("\nTurn Heaters Off")
    lab.Q1(0)
    lab.Q2(0)
    print("\nSet Heater 1:", lab.Q1(), "%",flush=True)
    print("Set Heater 2:", lab.Q2(), "%",flush=True)
    
    print("\nFinal Temperature 1: {0:0.2f} °C".format(lab.T1))
    print("Final Temperature 2: {0:0.2f} °C".format(lab.T2))

## Setting Maximum Heater Power

The control inputs to the heaters power is normally set with functions `.Q1()` and `.Q2()` (or properties `.U1` and `.U2`) specifying a value in a range from 0 to 100% of maximum heater power.  

The values of maximum heater power are specified in firmware with values in the range from 0 to 255. The default values are 200 for heater 1 and 100 for heater 2. The maximum heater power can be retrieved and set by properties `P1` and `P2`. The following code, for example, sets both heaters to a maximum power of 100.

In [ ]:
import tclab

with tclab.TCLab() as lab:
    print("Maximum power of heater 1 = ", lab.P1)
    print("Maximum power of heater 2 = ", lab.P2)
    
    print("Adjusting the maximum power of heater 1.")
    lab.P1 = 100
    
    print("Maximum power of heater 1 = ", lab.P1)
    print("Maximum power of heater 2 = ", lab.P2)

The actual power supplied to the heaters is a function of the power supply voltage applied to the Temperature Control Lab shield,

The maximum power applied to the heaters is a product of the settings (`P1`,`P2`) and of the power supply used with the TCLab hardware. The TCLab hardware is normally used with a 5 watt USB power supply capable of supply up to 1 amp at 5 volts. 

The TCLab hardware actually draws more than 1 amp when both `P1` and `P2` are set to 255 and `Q1` and `Q2` are at 100%. This situation will overload the power supply and result in the power supply shutting down. Normally the power supply will reset itself after unplugging from the power mains.

Experience with the device shows keeping the sum `P1` and `P2` to a value less than 300 will avoid problems with the 5 watt power supply. If you have access to larger power supplies, then you can adjust `P1` and `P2` accordingly to achieve a wider range of temperatures.

## `tclab` Sampling Speed

### Temperature Sampling Speed

In [ ]:
import time
import tclab

lab = tclab.setup(connected=True)

N = 100
meas = []
with lab() as a:
    tic = time.time()
    for k in range(0,N):
        meas.append(a.T1)
    toc = time.time()

print('Reading temperature at', round(N/(toc-tic),1), 'samples per second.')

### Heater Sampling Speed

In [ ]:
import time
import tclab

lab = tclab.setup(connected=True)

N = 100
with lab() as a:
    tic = time.time()
    for k in range(0,N):
        a.Q1(100)
    toc = time.time()

print('Setting heater at', round(N/(toc-tic),1), 'samples per second.')